In [6]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

from myfunc import text_clean #myfunc의 text_clean함수 가져오기

In [ ]:
?categoryNumber=001&pageNumber=2&pageSize=24 #24개짜리 한 페이지만 가져온 거임

In [4]:
#요청 & 파싱 파트 - 웹페이지를 파이썬으로 가져오는 과정
#24개짜리 한 페이지만 가져온 거임

url="https://www.yes24.com/product/category/bestseller"
payload=dict(categoryNumber='001',pageNumber=1,pageSize=24)
r=requests.get(url, params=payload)
soup = bs(r.content, 'lxml')
#soup #html yes24 페이지

soup = bs(r.content, 'lxml') -> HTML을 DOM 트리로 바꾸는 선언문 -> 부모–자식 관계 탐색 가능

- r.content : YES24 서버가 준 HTML 원본, 문자열 덩어리
- bs (BeautifulSoup) : HTML을 분석해주는 라이브러리, import해서 사용함
- lxml : HTML을 해석하는 엔진(parser) 이름, BeautifulSoup은 직접 해석 안 함 → 외주 줌, 부모–자식 구조로 정리

In [3]:
#soup.find_all("a") # a태그 결과 매우 많음

In [61]:
soup.select_one(".gd_name")
#정리된 DOM 트리에서 class가 gd_name인 처음 1개 요소 찾아줘

<a class="gd_name" href="/product/goods/172506508" onclick="wiseLogV2('BS', '001_005_001', ''); ">돈의 방정식</a>

In [62]:
soup.select_one(".gd_name").get_text()

'돈의 방정식'

In [10]:
soup.select_one("span.authPub.info_auth").get_text() #-> css
#span태그 안에 클래스가 2개(authPub.info_auth) 있는 것을 찾아서 가져와라

'\n이광수 저\r\n                            '

- css에서 셀렉터가 특히 중요함!!

### html, css 공부
문서 - 04폴더 들어가서 - cmd창 열고 - code . 쳐서 vs code 들어감 - 쓰고 alt+l+o 누르면 실행

### yes24 페이지 들어가서 -> f12 키 누르면 html창 나옴 -> 왼쪽 위에 있는 화살표 있는 버튼 클릭하면 마우스 갖다 댄 부분(제목, 저자 등..) html창 나옴

### 수집할 것들 -> 종류, 책 제목, 상세 페이지 링크, 저자/역/글그림/그림, 출판사, 출간일, 가격, 회원리뷰수, 평점

- soup : HTML 원문을 부모–자식 관계가 있는 DOM 구조로 바꿔 놓은 것
- select_one : DOM 트리에서 조건에 맞는 요소 “하나”를 찾아줘
- .get_text() : 태그 안의 글자만

```python
kind = text_clean(soup.select_one(".gd_res").get_text())
```
soup.select_one(".gd_res") → class가 gd_res인 요소 하나 찾아라

In [5]:
soup.select_one(".gd_res").get_text()

'[도서]'

In [49]:
#책 종류
kind=text_clean(soup.select_one(".gd_res").get_text())

#책 제목
book_title=soup.select_one(".gd_name").get_text()

# 책 상세 페이지 링크 구하기
#soup.select_one(".gd_name")
# 책 상세 페이지 주소 - 직접 가져옴
#https://www.yes24.com/product/goods/172506508

# 책 상세 페이지 링크
detail_link="https://www.yes24.com" + soup.select_one(".gd_name")['href']

#저자, 역자 함께 나옴 - 같은 태그에 클래스 2개
author=soup.select_one(".authPub.info_auth").get_text().replace("\n",'').replace("\r",'')

# 출판사
publisher=soup.select_one(".authPub.info_pub").get_text()

#출간일
pub_date=soup.select_one(".authPub.info_date").get_text()

#책 가격
# class가 info_row & info_price인 div 안에 있는 -> em 태그 중 class가 yes_b
book_price=int(soup.select_one("div.info_row.info_price em.yes_b").get_text().replace(",",''))

#리뷰 수
n_reviews=int(soup.select_one(".txC_blue").get_text())

#리뷰 평점
# class가 rating_grade인 태그 안에 있는 em 태그 중에서 class가 yes_b인 것
rating=float(soup.select_one(".rating_grade em.yes_b").get_text())

print(kind, book_title, detail_link, author, publisher, pub_date, book_price, n_reviews, rating)

도서 돈의 방정식 https://www.yes24.com/product/goods/172506508 모건 하우절 저/박영준 역                             서삼독 2026년 01월 25200 38 9.7


html돔트리 예시) 리뷰 평점

div.rating_grade        ← 부모
 └─ em.yes_b            ← 자식
     └─ "9.6"           ← 텍스트

In [52]:
# yesBestList의 자식 li만 필요함
soup.select("#yesBestList > li")[0] #yesBestList의 자식 li들 중 첫 번째 = 돈의 방정식

<li class="" data-goods-no="172506508" data-iy-no="0" data-statgb="02">
<div class="itemUnit">
<div class="item_img">
<div class="img_canvas">
<div class="img_upper">
<em class="ico rank">1</em>
<span class="rank_info rank_up">
<span class="ico"></span><em class="txt rank">3</em><em class="txt">위 상승</em>
</span>
</div>
<span class="img_item">
<span class="img_grp">
<a class="lnk_img" href="/product/goods/172506508" onclick="wiseLogV2('BS', '001_005_001', ''); ">
<em class="img_bdr">
<img alt="돈의 방정식" border="0" class="lazy" data-original="https://image.yes24.com/goods/172506508/L" src="https://image.yes24.com/momo/Noimg_L.jpg"/>
</em>
</a>
</span>
</span>
</div>
<div class="img_btn">
<a class="btnC btn_preview" href="javascript:yes24GU.openPreviewCheck(172506508); wiseLogV2('BS', '001_005_011', '');"><span class="bWrap"><em class="txt">미리보기</em></span></a>
</div>
</div>
<div class="item_info">
<div class="info_row info_keynote">
<span class="gd_keynote" id="spanGdKeynote">
<a class="ic

In [55]:
# 반복문으로 전체 출력

for item in soup.select("#yesBestList>li"):
    #책 종류
    kind=text_clean(item.select_one(".gd_res").get_text())

    #책 제목
    book_title=item.select_one(".gd_name").get_text()

    # 책 상세 페이지 링크
    detail_link="https://www.yes24.com" + item.select_one(".gd_name")['href']

    #저자, 역자 함께 나옴
    author=item.select_one(".authPub.info_auth").get_text().replace("\n",'').replace("\r",'')

    # 출판사
    publisher=item.select_one(".authPub.info_pub").get_text()

    #출간일
    pub_date=item.select_one(".authPub.info_date").get_text()

    #책 가격
    book_price=int(item.select_one("div.info_row.info_price em.yes_b").get_text().replace(",",''))

    #리뷰 수
    n_reviews=int(item.select_one(".txC_blue").get_text().replace(",",'')) if item.select_one(".txC_blue")!=None else 0

    #리뷰 평점
    rating=float(item.select_one(".rating_grade em.yes_b").get_text()) if item.select_one(".rating_grade em.yes_b")!=None else 0
    
    print(kind, book_title, detail_link, author, publisher, pub_date, book_price, n_reviews, rating)

도서 돈의 방정식 https://www.yes24.com/product/goods/172506508 모건 하우절 저/박영준 역                             서삼독 2026년 01월 25200 38 9.7
도서 괴테는 모든 것을 말했다 https://www.yes24.com/product/goods/164510819 스즈키 유이 저/이지수 역                             리프 2025년 11월 15300 193 9.2
도서 진보를 위한 주식투자 https://www.yes24.com/product/goods/168226997 이광수 저                             21세기북스 2025년 12월 19800 82 9.0
도서 엄마가 유령이 되었어! https://www.yes24.com/product/goods/27582352 노부미 글그림/이기웅 역                             길벗어린이 2016년 05월 10800 43 9.5
도서 최소한의 삼국지 https://www.yes24.com/product/goods/166390503 최태성 저/이성원 감수                             프런트페이지 2025년 11월 17550 185 9.8
도서 박곰희 연금 부자 수업 https://www.yes24.com/product/goods/148032819 박곰희 저                             인플루엔셜 2025년 06월 18900 293 9.7
도서 2026 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화(1,2,3급) https://www.yes24.com/product/goods/168393009 최태성 저                             이투스북  2025년 12월 19800 18 9.8
도서 자본주의 시대에서 살아남기 위한 최소한의 경제 공부 https://www.yes24.com/product/goods

In [57]:
soup.select_one(".gd_name")['href']

'/product/goods/172506508'

In [58]:
soup.select_one(".gd_name")['href'].split("/")[-1]

'172506508'

In [60]:
# 전체 모아서 데이터프레임(DataFrame)으로 출력하기

result={}

for item in soup.select("#yesBestList>li"):
    book_id=item.select_one(".gd_name")["href"].split("/")[-1]
    #책 종류
    kind=text_clean(item.select_one(".gd_res").get_text())

    #책 제목
    book_title=item.select_one(".gd_name").get_text()

    # 책 상세 페이지 링크
    detail_link="https://www.yes24.com" + item.select_one(".gd_name")['href']

    #저자, 역자 함께 나옴
    author=item.select_one(".authPub.info_auth").get_text().replace("\n",'').replace("\r",'')

    # 출판사
    publisher=item.select_one(".authPub.info_pub").get_text()

    #출간일
    pub_date=item.select_one(".authPub.info_date").get_text()

    #책 가격
    book_price=int(item.select_one("div.info_row.info_price em.yes_b").get_text().replace(",",''))

    #리뷰 수
    n_reviews=int(item.select_one(".txC_blue").get_text().replace(",",'')) if item.select_one(".txC_blue")!=None else 0

    #리뷰 평점
    rating=float(item.select_one(".rating_grade em.yes_b").get_text()) if item.select_one(".rating_grade em.yes_b")!=None else 0
    
    keys=("book_id", "kind", "book_title",'detail_link', 'author', 'publisher', 'pub_date', 'book_price','n_reviews', 'rating')
    values=(book_id,kind, book_title, detail_link, author, publisher, pub_date, book_price, n_reviews, rating)
    
    for key, value in zip(keys,values):
        result.setdefault(key, []).append(value)
        
df=pd.DataFrame(result)
df

    

,book_id,kind,book_title,detail_link,author,publisher,pub_date,book_price,n_reviews,rating
0,172506508,도서,돈의 방정식,https://www.yes24.com/product/goods/172506508,모건 하우절 저/박영준 역,서삼독,2026년 01월,25200,38,9.7
1,164510819,도서,괴테는 모든 것을 말했다,https://www.yes24.com/product/goods/164510819,스즈키 유이 저/이지수 역,리프,2025년 11월,15300,193,9.2
2,168226997,도서,진보를 위한 주식투자,https://www.yes24.com/product/goods/168226997,이광수 저,21세기북스,2025년 12월,19800,82,9.0
3,27582352,도서,엄마가 유령이 되었어!,https://www.yes24.com/product/goods/27582352,노부미 글그림/이기웅 역,길벗어린이,2016년 05월,10800,43,9.5
4,166390503,도서,최소한의 삼국지,https://www.yes24.com/product/goods/166390503,최태성 저/이성원 감수,프런트페이지,2025년 11월,17550,185,9.8
5,148032819,도서,박곰희 연금 부자 수업,https://www.yes24.com/product/goods/148032819,박곰희 저,인플루엔셜,2025년 06월,18900,293,9.7
6,168393009,도서,"2026 큰별쌤 최태성의 별별한국사 기출 500제 한국사능력검정시험 심화(1,2,3급)",https://www.yes24.com/product/goods/168393009,최태성 저,이투스북,2025년 12월,19800,18,9.8
7,167503385,도서,자본주의 시대에서 살아남기 위한 최소한의 경제 공부,https://www.yes24.com/product/goods/167503385,백억남(김욱현) 저,하이스트,2025년 12월,25200,43,9.7
8,168776448,도서,ETS 토익 정기시험 기출문제집 1000 Vol. 5 RC,https://www.yes24.com/product/goods/168776448,ETS 저,YBM(와이비엠),2025년 12월,19800,38,9.9
9,147276346,도서,너를 아끼며 살아라,https://www.yes24.com/product/goods/147276346,나태주 저,더블북,2025년 06월,16650,81,9.8


# yes24 종합 베스트 999권 자료 수집해서 csv파일로 저장하기
- book_data/
- yes24_수집일_best.csv
- 수집일은 datatime에서 오늘 날짜 추출해서 파일명에 포함시키기

# yes24_수집일_best.csv 파일에서 상세 링크만 가져와서 상세 페이지에 방문하고
- book_id
- 쪽수, 무게, 크기 (각각 따로 수집)
- 카테고리 분류 까지 모으기
- yes24_수집일_best_detail.csv 로 저장하기